# Getting Start

This chapter the basic usage of BboxTools. The BboxTools works on bounding box object which mainly store the information about a bounding box on a 2d image (could be in numpy.ndarry or torch.Tensor).

In [1]:
import BboxTools as bbt
import numpy as np

**Creating bbox** \
There are several way to create a bounding box object. You could use the basic way to create a Bbox. Print the bbox could give all the information about this bbox. \
Notice: Although it is possible to create a bbox without set the boundary of the image boundary, setting the image boundary when creating the box is always recommanded.

In [2]:
# Create a bbox with y: 2 to 5, x: 3 to 6, desire to working with image with size 8 * 8 
box = bbt.Bbox2D([(2, 5), (3, 6)], image_boundary=(8, 8))
print(box)

<class "Bbox2D", shape=[(2, 5), (3, 6)], boundary=[8, 8]>


**Bbox attributes** \
For a bbox, there are sereval useful attributes. All the attributes can be simple calculate by bbox.bbox.

In [3]:
print('bbox: ', box.bbox)  # The bounding box value, return in order [(y0, y1), (x0, x1)].
print('shape: ', box.shape)  # Shape of bounding box, return in (y_len, x_len).
print('size: ', box.size)  # Area covered box the box, as y_len * x_len.
print('boundary: ', box.boundary)  # The target boundary (image or array) of the bounding box, set when created. Return None if not set.
print('center: ', box.center)  # Center of the box, ((y0 + y1) / 2, (x0 + x1) / 2)
print('four points: ', box.lu, box.ru, box.rb, box.lb)  # Four corners (left top, right top, right bottom, left bottom)

bbox:  [[2, 5], [3, 6]]
shape:  (3, 3)
size:  9
boundary:  [8, 8]
center:  (3, 4)
four points:  (2, 3) (5, 3) (5, 6) (2, 6)


**Apply and assign bbox** \
Applying a bbox meaning crop the part inside the bbox for the whole image. Currently, bbox can be apply to numpy.ndarry (with shape (w, h) or (w, h, c)) and torch.Tensor (this will be detailly discribed in section ) 

In [4]:
# Create an 8 * 8 array with incremental value alone both x and y axis
a = np.arange(8).reshape((1, 8)) + np.arange(8).reshape((8, 1))
a

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 2,  3,  4,  5,  6,  7,  8,  9],
       [ 3,  4,  5,  6,  7,  8,  9, 10],
       [ 4,  5,  6,  7,  8,  9, 10, 11],
       [ 5,  6,  7,  8,  9, 10, 11, 12],
       [ 6,  7,  8,  9, 10, 11, 12, 13],
       [ 7,  8,  9, 10, 11, 12, 13, 14]])

In [5]:
# Apply this box on array a, which has same effects as a[2:5, 3:6]
print(box.apply(a))
assert np.all(box.apply(a) == a[2:5, 3:6])

[[5 6 7]
 [6 7 8]
 [7 8 9]]


Assign a bbox means filling the part inside the bbox with given value. The value could be int or float or array value. \
Notice: this function is a inplace function and return nothing.

In [6]:
box.assign(a, -1)
print(a)

[[ 0  1  2  3  4  5  6  7]
 [ 1  2  3  4  5  6  7  8]
 [ 2  3  4 -1 -1 -1  8  9]
 [ 3  4  5 -1 -1 -1  9 10]
 [ 4  5  6 -1 -1 -1 10 11]
 [ 5  6  7  8  9 10 11 12]
 [ 6  7  8  9 10 11 12 13]
 [ 7  8  9 10 11 12 13 14]]


In [7]:
box.assign(a, np.ones(box.shape) * -2)
print(a)

[[ 0  1  2  3  4  5  6  7]
 [ 1  2  3  4  5  6  7  8]
 [ 2  3  4 -2 -2 -2  8  9]
 [ 3  4  5 -2 -2 -2  9 10]
 [ 4  5  6 -2 -2 -2 10 11]
 [ 5  6  7  8  9 10 11 12]
 [ 6  7  8  9 10 11 12 13]
 [ 7  8  9 10 11 12 13 14]]


**Single box operations** \
Bbox support bbox operations including padding, shifting, transposing, scaling etc. \
Notice: most of the single operation (include pad, shift, etc) are not inplace.

In [8]:
# Pad the bounding box by 1 
print(box.pad(1))
# Pad the bounding box by (1, -1), which means, pad 1 on axis0 and -1 on axis1
print(box.pad((1, -1), axis=(0, 1)))

<class "Bbox2D", shape=[(1, 6), (2, 7)], boundary=[8, 8]>
<class "Bbox2D", shape=[(1, 6), (4, 5)], boundary=[8, 8]>


In [9]:
# Shift the box by (0, 1) and (1, 3)
print(box)
print(box.shift((0, 1)))

# Notice that when force sets to False (default) and the bbox has a boundary the bbox will remain inside the target with a unchanged shape
print(box.shift((1, 3)))

# If set force=True the returned bbox will be cropped when out of boundary (which will return the box with a different shape)
print(box.shift((1, 3), force=True))

<class "Bbox2D", shape=[(2, 5), (3, 6)], boundary=[8, 8]>
<class "Bbox2D", shape=[(2, 5), (4, 7)], boundary=[8, 8]>
<class "Bbox2D", shape=[(3, 6), (5, 8)], boundary=[8, 8]>
<class "Bbox2D", shape=[(3, 6), (6, 8)], boundary=[8, 8]>


In [10]:
# Transpose the bounding box (the bounding box will also be transposed)
print(box.transpose())

<class "Bbox2D", shape=[(3, 6), (2, 5)], boundary=[8, 8]>


In [11]:
# Scale the bounding box by given ratio. The ratio could be int or float or (int, int) or (float, float)
# Notice: boundary will also be modified and the value will be round to int.
print(box * 2)
print(box * (1.5, 2))

<class "Bbox2D", shape=[(4, 10), (6, 12)], boundary=[16, 16]>
<class "Bbox2D", shape=[(3, 7), (6, 12)], boundary=[12, 16]>


**Dual box operations** \
Dual box operations includes equal, intersection, and in-box and out-box operations. 

In [12]:
# Create two bbox on same image.
box_a = bbt.Bbox2D([(1, 6), (2, 7)], image_boundary=(8, 10))
box_b = bbt.Bbox2D([(3, 5), (4, 9)], image_boundary=(8, 10))

In [13]:
# To check if two bboxs are equal, using "==" 
# box_a == box_b only if the box_a.bbox and box_b.bbox is identical. (boundary is not necessary to be identical)
print(box_a == box_a)
print(box_a == box_b)

True
False


In [14]:
# To get the intersection of two bboxs, using "&"
print(box_a & box_b)

# Given a array view to demostrate the intersection operation
b = np.zeros((8, 10), dtype=np.uint8)
box_a.assign(b, 1)
box_b.assign(b, 2)
(box_a & box_b).assign(b, 3)
print(b)

<class "Bbox2D", shape=[(3, 5), (4, 7)], boundary=[8, 10]>
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 0 0 0]
 [0 0 1 1 1 1 1 0 0 0]
 [0 0 1 1 3 3 3 2 2 0]
 [0 0 1 1 3 3 3 2 2 0]
 [0 0 1 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


The box-in-box operation requires box_b to be inside box_a. And the operations will return a new bbox which is related bbox inside box_a with image_boundary = box_a.shape. \
box-out-box is the inverse operation of box_in_box.

In [15]:
# Create a box_b inside box_a
box_b = bbt.Bbox2D([(3, 5), (4, 7)], image_boundary=(8, 10))
box_in = box_a.box_in_box(box_b)
print(box_in)

<class "Bbox2D", shape=[(2, 4), (2, 5)], boundary=[5, 5]>


In [16]:
box_out = box_a.box_out_box(box_in)
print(box_out)

<class "Bbox2D", shape=[(3, 5), (4, 7)], boundary=[8, 10]>
